In [9]:
import os

os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_TRACING"] = "true"

In [10]:
from langchain.chains import MultiPromptChain, SequentialChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain_core.output_parsers import StrOutputParser

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [11]:
# Primeira SequentialChain: Geração de nome de empresa e slogan

# Passo 1: Gerar nome de empresa com base na descrição
nome_empresa_prompt = PromptTemplate(
    input_variables=["input"],
    template="Com base na descrição: '{input}', sugira um nome para uma empresa."
)

nome_empresa_chain = LLMChain(
    llm=llm, 
    prompt=nome_empresa_prompt,
    output_key="nome"
)

# Passo 2: Gerar slogan com base no nome da empresa
slogan_empresa_prompt = PromptTemplate(
    input_variables=["nome"],
    template="Com base no nome da empresa '{nome}', sugira um slogan para uma campanha de propaganda."
)

slogan_empresa_chain = LLMChain(
    llm=llm, 
    prompt=slogan_empresa_prompt,
    output_key="slogan"
)

# Definindo a primeira SequentialChain para empresas
empresa_chain = SequentialChain(
    chains=[nome_empresa_chain, slogan_empresa_chain],
    input_variables=["input"],
    output_variables=["nome", "slogan"]
)

In [12]:
# Segunda SequentialChain: Geração de nome de escola e slogan

# Passo 1: Gerar nome de escola com base na descrição
nome_escola_prompt = PromptTemplate(
    input_variables=["input"],
    template="Com base na descrição: '{input}', sugira um nome para uma escola."
)
nome_escola_chain = LLMChain(
    llm=llm, 
    prompt=nome_escola_prompt,
    output_key="nome"
)

# Passo 2: Gerar slogan com base no nome da escola
slogan_escola_prompt = PromptTemplate(
    input_variables=["nome"],
    template="Com base no nome da escola '{nome}', sugira um slogan para uma campanha de propaganda."
)
slogan_escola_chain = LLMChain(
    llm=llm, 
    prompt=slogan_escola_prompt, 
    output_key="slogan"
)

# Definindo a segunda SequentialChain para escolas
escola_chain = SequentialChain(
    chains=[nome_escola_chain, slogan_escola_chain],
    input_variables=["input"],
    output_variables=["nome", "slogan"]
)

In [13]:
default_prompt = PromptTemplate(
    input_variables=["input"],
    template="Dado um prompt a seguir ```{input}```, responda"
)

default_chain = default_prompt | llm | StrOutputParser()

In [14]:
MULTI_PROMPT_ROUTER_TEMPLATE = r"""
Dado um texto de entrada, seleciona o prompt que melhor se encaixa para a entrada.
Você será provido com um nome e uma descrição de situação em que o prompt deve ser utilizado.

<< Formatação >>
Retorne um código markdown com um objeto JSON formatado da seguinte forma:

```json
{{{{
    "destination": string \ nome do prompt usado ou "DEFAULT"
    "next_inputs": string \ o prompt de entrada
}}}}
```

<< PROMPTS CANDIDATOS >>
{destinations}

<< ENTRADA >>
{{input}}

<< OUTPUT (lembre de incluir o ```json)>>"""

destinations = """
'empresa: bom para responder sobre empresas\nescola: bom para responder sobre escolas'
"""

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt, verbose=True)

In [15]:
destination_chains = {}
destination_chains['empresa'] = empresa_chain
destination_chains['escola'] = escola_chain

In [19]:
from operator import itemgetter
from typing import Literal
from typing_extensions import TypedDict
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def return_chain(x):
    
    if x["destination"] == "empresa":
        return(empresa_chain)
    
    if x['destination'] == "escola":
        return(escola_chain)
    
    return(default_chain)

class RouteQuery(TypedDict):
    """Route query to destination."""
    destination: Literal["empresa", "escola", "default"]

route_chain = (
    router_prompt
    | llm.with_structured_output(RouteQuery)
    | itemgetter("destination")
)

multi_prompt_chain = {
    "destination": route_chain,
    "input": lambda x: x["input"],
} | RunnableLambda(
    lambda x: return_chain(x),
)

In [20]:
# Testando a estrutura
descricao_empresa = "Uma empresa de tecnologia focada em inteligência artificial."
descricao_escola = "Uma escola voltada para educação infantil e fundamental."
descricao_outra_empresa = "Um novo produto para jardinagem."
descricao_outro = "Rosas são vermelhas, violetas são azuis"

# Chamadas de teste
print(multi_prompt_chain.invoke({"input":descricao_empresa}))
print(multi_prompt_chain.invoke({"input":descricao_escola}))
print(multi_prompt_chain.invoke({"input":descricao_outra_empresa}))
print(multi_prompt_chain.invoke({"input":descricao_outro}))

{'destination': 'empresa', 'input': 'Uma empresa de tecnologia focada em inteligência artificial.', 'nome': 'Aqui estão algumas sugestões de nomes para uma empresa de tecnologia focada em inteligência artificial:\n\n1. **Aurora AI**: "Aurora" sugere uma nova era de iluminação e conhecimento, enquanto "AI" destaca a especialização em inteligência artificial.\n2. **CerebroX**: "Cerebro" é uma referência clara à inteligência e ao processamento de informações, enquanto "X" sugere uma abordagem inovadora e experimental.\n3. **MindSpark**: "Mind" (mente) e "Spark" (faísca) sugerem a geração de ideias e soluções inovadoras através da inteligência artificial.\n4. **Nexa Inteligência**: "Nexa" sugere conexão e integração, enquanto "Inteligência" destaca a especialização em IA.\n5. **Pulse AI**: "Pulse" (pulso) sugere energia e vitalidade, enquanto "AI" destaca a especialização em inteligência artificial.\n6. **Synapse Tecnologia**: "Synapse" (sinapse) é uma referência à conexão entre neurônios,